In [2]:
import os

import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
from astropy.table import Table
from itertools import combinations
from astropy.coordinates import SkyCoord
from astropy import units as u


save = True

sns.set(style="whitegrid")

In [38]:
from astropy.coordinates import SkyCoord
from astropy import units as u
import numpy as np


def compute_full_orbital_elements(star1, star2):
    G = 4.302e-3  # pc * (km/s)^2 / Msun

    # Create SkyCoord objects
    c1 = SkyCoord(
        ra=star1['ra'] * u.deg,
        dec=star1['dec'] * u.deg,
        distance=(1000 / star1['parallax']) * u.pc,
        pm_ra_cosdec=star1['pmra'] * u.mas/u.yr,
        pm_dec=star1['pmdec'] * u.mas/u.yr,
        radial_velocity=star1['radial_velocity'] * u.km/u.s
    )

    c2 = SkyCoord(
        ra=star2['ra'] * u.deg,
        dec=star2['dec'] * u.deg,
        distance=(1000 / star2['parallax']) * u.pc,
        pm_ra_cosdec=star2['pmra'] * u.mas/u.yr,
        pm_dec=star2['pmdec'] * u.mas/u.yr,
        radial_velocity=star2['radial_velocity'] * u.km/u.s
    )

    c_bulk = SkyCoord(
        ra= 56.74 * u.deg,    # this two are the same coordinate used for Gaia ADQL
        dec= 24.09 * u.deg,
        distance= 1000 / 7.316455 * u.pc,
        pm_ra_cosdec= 19.783799 * u.mas/u.yr,
        pm_dec= -45.080216 * u.mas/u.yr,
        radial_velocity= 6.861443 * u.km/u.s
    )
    
    v_bulk = c_bulk.velocity.d_xyz.to(u.km/u.s).value  # 3D vector of cluster motion

    print(f"Bulk motion: {v_bulk} km/s and {np.linalg.norm(v_bulk):.2f} km/s")

    r1 = c1.cartesian.xyz.to(u.pc).value
    v1 = c1.velocity.d_xyz.to(u.km/u.s).value #- v_bulk
    r2 = c2.cartesian.xyz.to(u.pc).value
    v2 = c2.velocity.d_xyz.to(u.km/u.s).value #- v_bulk

    # convert into CM frame
    m1 = star1['mass']
    m2 = star2['mass']
    M = m1 + m2
    M_r = (m1*m2)/(m1+m2)

    r_com = (m1 * r1 + m2 * r2) / M  # pc
    v_com = (m1 * v1 + m2 * v2) / M  # km/s

    r1_com = r1 - r_com
    r2_com = r2 - r_com

    print(f"check position in the CM frame (sould be zero): {np.linalg.norm(m1*r1_com + m2*r2_com):.2f} pc")

    v1_com = v1 - v_com
    v2_com = v2 - v_com

    print(f"check velocity in the CM frame: {v1_com, v2_com} km/s")

    r_rel = r2_com - r1_com
    v_rel = v2_com - v1_com
    r_mag = np.linalg.norm(r_rel)
    v_mag = np.linalg.norm(v_rel)
    v1_mag = np.linalg.norm(v1_com)
    v2_mag = np.linalg.norm(v2_com)

    h_vec = np.cross(r_rel, v_rel)
    h_mag = np.linalg.norm(h_vec)

    print(f"angular momentum vector: {h_vec} (magnitude {h_mag:.3f} pc km/s)")
    print(f"check angular momentum conservation: {np.linalg.norm(m1 * h_vec + m2 * h_vec):.3f} pc km/s")
    print(f"check prod of r and h: {np.dot(r_rel, h_vec):.3f} pc km/s (should be zero)")

    E = 0.5*M_r*v_mag**2 - G*(m1*m2)/r_mag
    a = -G * M / (2 * E)

    # velocities if the stars were in a circulr orbit at distance r_mag
    v_circ = np.sqrt(G * M / r_mag)
    print(f"check circular velocity: {v_circ:.2f} km/s (should be close to {v1_mag:.2f} km/s and {v2_mag:.2f} km/s)")
    
    # Now let's see the second version:
    mu = G*(m1+m2)
    C = v_mag**2 / 2 - mu / r_mag
    E2 = C*M_r

    a = - mu/(2*C)

    print(f"check energy: {E:.5f} vs {E2:.5f}")

    e_vec = (v_mag**2. - mu/r_mag) * r_rel - np.dot(r_rel,v_rel) * v_rel / mu
    e = np.linalg.norm(e_vec)

    i_rad = np.arccos(h_vec[2] / h_mag)
    i_deg = np.degrees(i_rad)

    # Node vector (points toward ascending node)
    K = np.array([0, 0, 1])
    n_vec = np.cross(K, h_vec)
    n_mag = np.linalg.norm(n_vec)

    # Longitude of ascending node Ω
    Omega = np.degrees(np.arccos(n_vec[0] / n_mag)) if n_mag != 0 else 0
    if n_vec[1] < 0:
        Omega = 360 - Omega

    # Argument of periapsis ω
    if n_mag != 0 and e != 0:
        omega = np.degrees(np.arccos(np.dot(n_vec, e_vec) / (n_mag * e)))
        if e_vec[2] < 0:
            omega = 360 - omega
    else:
        omega = 0

    # True anomaly ν
    if e != 0:
        nu = np.degrees(np.arccos(np.dot(e_vec, r_rel) / (e * r_mag)))
        if np.dot(r_rel, v_rel) < 0:
            nu = 360 - nu
    else:
        nu = 0

    T_yr = 2 * np.pi * np.sqrt(a**3 / (G * M))

    print(f"Relative velocity: {v_mag:.2f} km/s")
    print(f"Relative separation: {r_mag:.3f} pc")
    print(f"Total energy: {E:.5f}")


    return {
        'semi_major_axis_pc': a,
        'eccentricity': e,
        'inclination_deg': i_deg,
        'longitude_of_ascending_node_deg': Omega,
        'argument_of_periapsis_deg': omega,
        'true_anomaly_deg': nu,
        'orbital_period_yr': T_yr
    }

example_star1 = {
    'ra': 57.491184,
    'dec': 23.847947,
    'parallax': 7.361154,              # Distance ~133.3 pc
    'pmra': 21.026666,                 # mas/yr
    'pmdec': -48.132843,               # mas/yr
    'radial_velocity': 4.707757,       # km/s
    'mass': 5.0                        # Msun
}

example_star2 = {
    'ra': 57.489468,                   # ~3.6 arcsec east = ~0.0023 pc
    'dec': 23.846882,                  # ~3.6 arcsec north = ~0.0023 pc
    'parallax': 7.642781,              # Same distance                        
    'pmra': 21.530787,                 # Slight relative tangential motion
    'pmdec': -43.565036,
    'radial_velocity': 0.934193,       # Small RV difference
    'mass': 1.067
}

full_orbit_params = compute_full_orbital_elements(example_star1, example_star2)
full_orbit_params


Bulk motion: [ -0.74470962  22.23688846 -23.86380014] km/s and 32.63 km/s
check position in the CM frame (sould be zero): 0.00 pc
check velocity in the CM frame: (array([ 0.45045831,  0.76791202, -0.3710974 ]), array([-2.11086367, -3.59846307,  1.73897565])) km/s
angular momentum vector: [-16.99629172  10.37413518   0.83617474] (magnitude 19.930 pc km/s)
check angular momentum conservation: 120.914 pc km/s
check prod of r and h: 0.000 pc km/s (should be zero)
check circular velocity: 0.07 km/s (should be close to 0.96 km/s and 4.52 km/s)
check energy: 13.21992 vs 13.21992
Relative velocity: 5.48 km/s
Relative separation: 5.006 pc
Total energy: 13.21992


C:\Users\bosca\AppData\Local\Temp\ipykernel_17308\593023750.py:127: RuntimeWarning: invalid value encountered in sqrt
  T_yr = 2 * np.pi * np.sqrt(a**3 / (G * M))


{'semi_major_axis_pc': -0.0008680523071245273,
 'eccentricity': 3865.5163820671546,
 'inclination_deg': 87.59538889080832,
 'longitude_of_ascending_node_deg': 238.60109437786934,
 'argument_of_periapsis_deg': 204.26899368895488,
 'true_anomaly_deg': 131.83287605286384,
 'orbital_period_yr': nan}

In [ ]:
from astropy.coordinates import SkyCoord
from astropy import units as u
import numpy as np

def compute_orbital_elements_from_phase_space(star1, star2):
    G = 4.302e-3  # pc * (km/s)^2 / Msun

    # Average parallax to enforce consistent distance scale
    avg_parallax = (star1['parallax'] + star2['parallax']) / 2  # mas
    distance = (1000 / avg_parallax) * u.pc

    # SkyCoord objects
    c1 = SkyCoord(
        ra=star1['ra'] * u.deg,
        dec=star1['dec'] * u.deg,
        distance=distance,
        pm_ra_cosdec=star1['pmra'] * u.mas/u.yr,
        pm_dec=star1['pmdec'] * u.mas/u.yr,
        radial_velocity=star1['radial_velocity'] * u.km/u.s
    )

    c2 = SkyCoord(
        ra=star2['ra'] * u.deg,
        dec=star2['dec'] * u.deg,
        distance=distance,
        pm_ra_cosdec=star2['pmra'] * u.mas/u.yr,
        pm_dec=star2['pmdec'] * u.mas/u.yr,
        radial_velocity=star2['radial_velocity'] * u.km/u.s
    )

    # Masses and CM quantities
    m1, m2 = star1['mass'], star2['mass']
    M = m1 + m2
    mu = (m1 * m2) / M

    r1 = c1.cartesian.xyz.to(u.pc).value
    v1 = c1.velocity.d_xyz.to(u.km/u.s).value
    r2 = c2.cartesian.xyz.to(u.pc).value
    v2 = c2.velocity.d_xyz.to(u.km/u.s).value

    # Relative position and velocity in CM frame
    r_rel = r2 - r1
    v_rel = v2 - v1
    r_mag = np.linalg.norm(r_rel)
    v_mag = np.linalg.norm(v_rel)

    # Angular momentum
    h_vec = np.cross(r_rel, v_rel)
    h_mag = np.linalg.norm(h_vec)

    # Specific mechanical energy
    E = 0.5 * v_mag**2 - G * M / r_mag
    a = -G * M / (2 * E)  # semi-major axis

    # Eccentricity vector
    e_vec = (np.cross(v_rel, h_vec) / (G * M)) - (r_rel / r_mag)
    e = np.linalg.norm(e_vec)

    # Inclination
    i_rad = np.arccos(h_vec[2] / h_mag)
    i_deg = np.degrees(i_rad)

    # Node vector
    K = np.array([0, 0, 1])
    n_vec = np.cross(K, h_vec)
    n_mag = np.linalg.norm(n_vec)

    # Longitude of ascending node
    if n_mag != 0:
        Omega = np.degrees(np.arccos(n_vec[0] / n_mag))
        if n_vec[1] < 0:
            Omega = 360 - Omega
    else:
        Omega = 0

    # Argument of periapsis
    if n_mag != 0 and e > 1e-8:
        omega = np.degrees(np.arccos(np.dot(n_vec, e_vec) / (n_mag * e)))
        if e_vec[2] < 0:
            omega = 360 - omega
    else:
        omega = 0

    # True anomaly
    if e > 1e-8:
        nu = np.degrees(np.arccos(np.dot(e_vec, r_rel) / (e * r_mag)))
        if np.dot(r_rel, v_rel) < 0:
            nu = 360 - nu
    else:
        nu = 0

    # Orbital period (years)
    T_yr = 2 * np.pi * np.sqrt((a**3) / (G * M))

    return {
        'semi_major_axis_pc': a,
        'eccentricity': e,
        'inclination_deg': i_deg,
        'longitude_of_ascending_node_deg': Omega,
        'argument_of_periapsis_deg': omega,
        'true_anomaly_deg': nu,
        'orbital_period_yr': T_yr
    }


,r_mag_pc,v_mag_kms,v_esc_kms,angle_deg
Simulated Binary,0.010000,1.136046,1.606611,90.000000
Gaia Binary Candidate,0.004413,4.761537,3.439414,113.305341
